<a href="https://colab.research.google.com/github/sarodriguezva/Paralela-2022/blob/main/BlurVideo/src/BlurVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!../usr/local/cuda/bin/nvcc --version

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
#@title Install OpenCV
!apt-get install -qq gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 
!ln -s /usr/bin/g++-5 

!sudo apt-get update
!sudo apt-get upgrade

#Install Dependencies
!sudo apt-get install -y build-essential 
!sudo apt-get install -y cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev
#The following command is needed to process images:
!sudo apt-get install -y python-dev python-numpy libtbb2 libtbb-dev libjpeg-dev libpng-dev libtiff-dev libjasper-dev libdc1394-22-dev
#To process videos:
!sudo apt-get install -y libavcodec-dev libavformat-dev libswscale-dev libv4l-dev
!sudo apt-get install -y libxvidcore-dev libx264-dev
#For GUI:
!sudo apt-get install -y libgtk-3-dev
#For optimization:
!sudo apt-get install -y libatlas-base-dev gfortran pylint

In [ ]:
#@title Install OpenCV P.2.
!wget https://github.com/opencv/opencv/archive/4.6.0.zip -O opencv-4.6.0.zip
!sudo apt-get install unzip
!unzip opencv-4.6.0.zip
!wget https://github.com/opencv/opencv_contrib/archive/4.6.0.zip -O opencv_contrib.zip
!sudo apt-get install unzip
!unzip opencv_contrib.zip

In [ ]:
!sudo rm -rf build

In [ ]:
%cd ..

/content


In [ ]:
#@title Install OpenCV P.3.
%cd opencv-4.6.0
!mkdir build
%cd build
!cmake -D OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib-4.6.0/modules -D WITH_TBB=ON -D WITH_FFMPEG=ON -D WITH_GTK=ON -D WITH_V4L=ON -D WITH_OPENGL=ON -D WITH_CUBLAS=ON -DWITH_QT=OFF -D WITH_OPENMP=ON -D WITH_IPP=ON -D CMAKE_BUILD_TYPE=RELEASE -D BUILD_EXAMPLES=OFF -DCUDA_NVCC_FLAGS="-D_FORCE_INLINES" -D WITH_NVCUVID=ON -D WITH_CUDA=ON -D BUILD_DOCS=OFF -D BUILD_PERF_TESTS=OFF -D BUILD_TESTS=OFF -D WITH_CSTRIPES=ON -D WITH_OPENCL=ON CMAKE_INSTALL_PREFIX=/usr/local/ ..
!make -j`nproc`
!sudo make install

/content/opencv-4.6.0
/content/opencv-4.6.0/build
-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- ocv_init_download: OpenCV source tree is not fetched as git repository. 3rdparty resources will be downloaded from github.com by default.
-- Detected processor: x86_64
-- Found PythonInterp: /usr/bin/python2.7 (found suitable version "2.7.17", minimum required is "2.7") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython2.7.so (found suitable exact version "2.7.17") 
-- Found PythonInterp: /usr/bin/python3 (found suitable version "3.7

In [ ]:
#@title vector add

%%cu
/************* add vector ******************************************************/
#include <stdio.h>
// For the CUDA runtime routines (prefixed with "cuda_")
#include <cuda_runtime.h>

  
/*******************************************************************************/

__global__ void
vectorAdd(const float *A, const float *B, float *C, int numElements)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;

    if (i < numElements)
    {
        C[i] = A[i] + B[i];
    }
}

/*******************************************************************************/
int main(void)
{
    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Print the vector length to be used, and compute its size
    int numElements = 50000;
    size_t size = numElements * sizeof(float);
    printf("[Vector addition of %d elements]\n", numElements);

    // Allocate the host input vector A
    float *h_A = (float *)malloc(size);

    // Allocate the host input vector B
    float *h_B = (float *)malloc(size);

    // Allocate the host output vector C
    float *h_C = (float *)malloc(size);

    // Verify that allocations succeeded
    if (h_A == NULL || h_B == NULL || h_C == NULL)
    {
        fprintf(stderr, "Failed to allocate host vectors!\n");
        exit(EXIT_FAILURE);
    }

    // Initialize the host input vectors
    for (int i = 0; i < numElements; ++i)
    {
        h_A[i] = rand()/(float)RAND_MAX;
        h_B[i] = rand()/(float)RAND_MAX;
    }

    // Allocate the device input vector A
    float *d_A = NULL;
    err = cudaMalloc((void **)&d_A, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector A (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Allocate the device input vector B
    float *d_B = NULL;
    err = cudaMalloc((void **)&d_B, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector B (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Allocate the device output vector C
    float *d_C = NULL;
    err = cudaMalloc((void **)&d_C, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Copy the host input vectors A and B in host memory to the device input vectors in
    // device memory
    printf("Copy input data from the host memory to the CUDA device\n");
    err = cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector A from host to device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    err = cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector B from host to device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Launch the Vector Add CUDA Kernel
    int threadsPerBlock = 256;
    int blocksPerGrid =(numElements + threadsPerBlock - 1) / threadsPerBlock;
    printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, numElements);
    err = cudaGetLastError();

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to launch vectorAdd kernel (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Copy the device result vector in device memory to the host result vector
    // in host memory.
    printf("Copy output data from the CUDA device to the host memory\n");
    err = cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Verify that the result vector is correct
    for (int i = 0; i < numElements; ++i)
    {
        if (fabs(h_A[i] + h_B[i] - h_C[i]) > 1e-5)
        {
            fprintf(stderr, "Result verification failed at element %d!\n", i);
            exit(EXIT_FAILURE);
        }
    }

    // Free device global memory
    err = cudaFree(d_A);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector A (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
    err = cudaFree(d_B);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector B (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
    err = cudaFree(d_C);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    // Reset the device and exit
    err = cudaDeviceReset();

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to deinitialize the device! error=%s\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    printf("Done\n");
    return 0;
}

[Vector addition of 50000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 196 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Done



In [ ]:
#@title BlurVideo
%%cu

#include <opencv2/imgcodecs.hpp>
#include <opencv2/highgui.hpp>
#include <opencv2/imgproc.hpp>
#include <opencv2/objdetect.hpp>
#include <iostream>

using namespace cv;
using namespace std;

//Blur Radio - CONSTANT
int r = 10;

Mat summed_table(Mat ROI, int w, int h){
    Mat table;
    table = Mat::zeros(ROI.size(), CV_32SC3);

    table.at<Vec3i>(0,0) = ROI.at<Vec3i>(0,0);
    for (int x = 1; x < w; x++){
        table.at<Vec3i>(0,x) = ROI.at<Vec3i>(0,x) + table.at<Vec3i>(0,x-1);
    }

    for (int y = 1; y < h; y++){
        table.at<Vec3i>(y,0) = ROI.at<Vec3i>(y,0) + table.at<Vec3i>(y-1,0);
    }

    for (int y = 1; y < h-1; y++){
        for (int x = 1; x < w-1; x++){
            table.at<Vec3i>(y,x) = ROI.at<Vec3i>(y,x) + table.at<Vec3i>(y-1,x) + table.at<Vec3i>(y,x-1) - table.at<Vec3i>(y-1,x-1);
        }
    }

    return table;
}

void myBlur(Mat face, int w, int h){
    int area = pow(2*r+1, 2);
    
    Mat table = summed_table(face, w, h);

    for (int y = r+1; y < h-r-1; y++){
        for (int x = r+1; x < w-r-1; x++){
            face.at<Vec3i>(y,x) = table.at<Vec3i>(y+r, x+r) - table.at<Vec3i>(y+r, x-r-1) 
                                - table.at<Vec3i>(y-r-1, x+r) + table.at<Vec3i>(y-r-1, x-r-1);
            face.at<Vec3i>(y,x) /= area;
        }
    }
}

Rect setROI(Rect faceROI){
    Rect ROI = faceROI;
    
    if(ROI.x - r >= 0 && ROI.y - r >= 0 && ROI.x + ROI.width + 2*r <= 640 && ROI.y + ROI.height + 2*r <= 480){
        ROI.x -= r;
        ROI.y -= r;
        ROI.width += 2*r;
        ROI.height += 2*r;
    }
    
    return ROI;
}

int main(int argc, char *argv[]){
    int THREADS = 16;
    //Load Video
    //"../resources/videoIn.mp4"
    //string path = argv[1];
    string path = "./videoInShort.mp4";
    VideoCapture cap(path);

    // Check if camera opened successfully
    if (!cap.isOpened()){
        perror("Error opening video stream or file");
        exit(-1);
    }

    //Load haarcascade
    CascadeClassifier faceCascade;
    faceCascade.load("./haarcascade_frontalface_default.xml");

    if (faceCascade.empty()){
        perror("XML File not loaded");
        exit(-1);
    }
    
    //"../resources/videoOut.avi"
    //VideoWriter video(argv[2],cv::VideoWriter::fourcc('M','J','P','G'),10, Size(640,480));
    VideoWriter video("./videoOutShort.avi",cv::VideoWriter::fourcc('M','J','P','G'),10, Size(640,480));
    
    printf("Procesando video");
    while (true){
        Mat img;
        cap >> img;
        if (img.empty()){
            cout << "No frame captured from video" << endl;
            break;
        }

        resize(img, img, Size(640,480));

        Mat img_gray;
        cvtColor(img, img_gray, COLOR_BGR2GRAY);

        vector<Rect> faces;
        faceCascade.detectMultiScale(img_gray, faces, 1.1, 10);
        
        for (int i = 0; i < faces.size(); i++){
            img.convertTo(img, CV_32SC3);

            Rect R = setROI(faces[i]);

            //rectangle(img, faces[i].tl(), faces[i].br(), Scalar(0, 255,0), 3);
            
            Point top_left = R.tl();
            Point bot_right = R.br();

            int w = bot_right.x - top_left.x;
            int h = bot_right.y - top_left.y;

            //rectangle(img, top_left, bot_right, Scalar(0,0,0), 3);

            Mat faceROI = img(R);
            faceROI.convertTo(faceROI, CV_32SC3);

            myBlur(faceROI, w, h);
            img.convertTo(img, CV_8UC3);
        }

        video.write(img);
        //imshow("Blurred Face Detection", img);

        if (waitKey(1) == 'q'){
            break;
        }
    }

    printf("Video guardado con éxito");
    cap.release();
    video.release();

    destroyAllWindows();
    
    return 0;
}

//run program: /home/sarodriguezva/Proyectos/Paralela-2022/BlurVideo/build/BlurVideo ../resources/videoIn.mp4 ../resources/videoOut.avi 1
//run program (Generic): build/BlurVideo ../resources/videoIn.mp4 ../resources/videoOut.avi 1
//BlurVideo pathToVideoIn pathToVideoOut THREADS

/tmp/tmph3rr7kah/d28ad20e-7717-48a8-8afb-c1f34f80e5a0.cu(64): warning: variable "THREADS" was declared but never referenced

/tmp/tmpxft_0000059a_00000000-11_d28ad20e-7717-48a8-8afb-c1f34f80e5a0.o: In function `summed_table(cv::Mat, int, int)':
tmpxft_0000059a_00000000-6_d28ad20e-7717-48a8-8afb-c1f34f80e5a0.cudafe1.cpp:(.text+0xd7): undefined reference to `cv::Mat::zeros(cv::Size_<int>, int)'
/tmp/tmpxft_0000059a_00000000-11_d28ad20e-7717-48a8-8afb-c1f34f80e5a0.o: In function `main':
tmpxft_0000059a_00000000-6_d28ad20e-7717-48a8-8afb-c1f34f80e5a0.cudafe1.cpp:(.text+0x906): undefined reference to `cv::VideoCapture::VideoCapture(cv::String const&)'
tmpxft_0000059a_00000000-6_d28ad20e-7717-48a8-8afb-c1f34f80e5a0.cudafe1.cpp:(.text+0x924): undefined reference to `cv::VideoCapture::isOpened() const'
tmpxft_0000059a_00000000-6_d28ad20e-7717-48a8-8afb-c1f34f80e5a0.cudafe1.cpp:(.text+0x950): undefined reference to `cv::CascadeClassifier::CascadeClassifier()'
tmpxft_0000059a_00000000-6_d28ad20e